Object Detection-RCNN系列
===

#1. RCNN

##1.1 RCNN基本过程
- 先将region通过ss检测出来(2k+)，然后根据cnn提取的region特征丢入svm进行分类(compute score)，得到的就是一个region-bbox以及对应的类别
- 再利用(IoU->nms)得到具体的框，目的防止泛滥，为了精确bbox.
- 再根据pool5 feature做了个bbox regression来decrease location error- 其中training的trick则为hnm＋finetuning)

##1.2 训练过程


###1.2.1 预训练
首先需要训练一个完整的alexnet网络，用于生成后续网络的预训练模型

###1.2.2 数据预处理
输入的数据都是带有bounding box(ground true)信息的图片，对于每张图片都需要做如下处理：
- 使用selectivesearch.selective_search生成若干的bounding box
- 通过算法合并和过滤一些bounding box
- 计算每个bounding box和ground true的IOU，如果大于阀值(一般为0.3)，那么这个bounding box对应的label就是这张图片的label(正样本)；否则对应的label就是背景label(负样本)
- 这样一张图片就可以生成若干的记录，每个记录包括bounding box截取的图片信息、对应的label以及bounding box的坐标信息
- 这些bounding box对应的图片信息，需要resize到224*224(alexnet的输入格式)
- 这样可以生成很多很多图片数据。每批数据应该包含128个实例，其中32个正样本，96个负样本

###1.2.3 构建新网络
构建一个与之前的alexnet一模一样的网络，只不过需要输出的4096维的特征向量，而不是之前的具体分类个数的特征向量，然后加载之前训练好的模型参数。然后将数据预处理中的图片信息输入到这个网络进行预测，可以得到若干的特征向量，每个特征向量是4096维的。将这些特征向量与对应的label输入到SVM.LinearSVC模型进行训练

###1.2.4 预测
将需要预测的图片通过selectivesearch.selective_search生成若干的bounding box,然后放入2.3中的新网络生成若干的4096维特征向量，然后将这些特征向量放入已经已经训练好的SVM模型中进行预测，如果结果是背景，那么舍弃，否则这个bounding box就需要画出来。

###1.2.5 Bounding-box Regression边框回归
具体可以参考http://m.blog.csdn.net/zijin0802034/article/details/77685438，但是没有具体的代码实现

###1.2.6 总结
![image](images/01.png)

#2. Fast RCNN

##2.1 RCNN的缺陷
- RCNN一张图像内候选框之间大量重叠，提取特征操作冗余
- 由于提取特征操作冗余，所以训练时速度很慢
- RCNN中独立的分类器和回归器需要大量特征作为训练样本，所以训练时所需空间较大


##2.2 Fast RCNN的改进
针对RCNN的问题，Fast RCNN从以下三个方面进行了改进：


###2.2.1 测试时速度慢 
RCNN一张图像内候选框之间大量重叠，提取特征操作冗余。 
本文将整张图像归一化后直接送入深度网络。在邻接时，才加入候选框信息，在末尾的少数几层处理每个候选框。


###2.2.2 训练时速度慢 
原因同上。 
在训练时，本文先将一张图像送入网络，紧接着送入从这幅图像上提取出的候选区域。这些候选区域的前几层特征不需要再重复计算。


###2.2.3 训练所需空间大 
RCNN中独立的分类器和回归器需要大量特征作为训练样本。 
把类别判断和位置精调统一用深度网络实现，不再需要额外存储。

##2.3 总结
RCNN主要是使用Selective Search算法计算出Proposal，然后送入SVM进行分类训练，然后送入一个CNN的回归网络进行bounding box的训练。而Fast RCNN仍然需要使用Selective Search算法计算出Proposal，只是它将后续的分类训练和回归训练合并到了一个CNN网络中，不在需要进行两步操作(ROI层)

#3. Faster RCNN
Faster RCNN在Fast RCNN的基础上，将proposal的选取工作也放到了网络中，这样通过一个CNN网络，就可以同时生成Proposal(RPN层)，而且也可以同时进行分类和回归的训练
![image](images/02.png)